## First quick start

### 导入torch库

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

In [3]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=False,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [5]:
batch_size = 128

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(X.shape)
    print(y.shape, y.dtype)
    break

torch.Size([128, 1, 28, 28])
torch.Size([128]) torch.int64


In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [7]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10) 
        )
    
    def forward(self, x):
        x = self.flatten(x)
        # 这里logits 意思是指的这个数据没有做softmax 变换
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [9]:
def train_epoch(train_dataloader, model, loss_fn, optimizer):
    model.train()
    for i, (X, y) in enumerate(train_dataloader):
        # 加载对硬件
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        # 更新参数
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
            print(f"batch {i} loss value: {loss.item()}")

In [10]:
def test_epoch(test_dataloader, model, loss_fn):
    model.eval()
    test_size = len(test_dataloader.dataset)
    acc = 0
    with torch.no_grad():
        for X, y in test_dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            loss = loss_fn(pred, y)

            acc += (pred.argmax(1) == y).type(torch.float).sum().item()

    acc /= test_size

    print(f"acc: {acc}")





In [11]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_epoch(train_dataloader, model, loss_fn, optimizer)
    test_epoch(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
batch 0 loss value: 2.2916154861450195
batch 100 loss value: 2.280522584915161
batch 200 loss value: 2.2730696201324463
batch 300 loss value: 2.262801170349121
batch 400 loss value: 2.242841958999634
acc: 0.3
Epoch 2
-------------------------------
batch 0 loss value: 2.227956533432007
batch 100 loss value: 2.209622859954834
batch 200 loss value: 2.2052741050720215
batch 300 loss value: 2.1957688331604004
batch 400 loss value: 2.16390323638916
acc: 0.4377
Epoch 3
-------------------------------
batch 0 loss value: 2.1463890075683594
batch 100 loss value: 2.1172001361846924
batch 200 loss value: 2.113025188446045
batch 300 loss value: 2.0993001461029053
batch 400 loss value: 2.0555522441864014
acc: 0.4913
Epoch 4
-------------------------------
batch 0 loss value: 2.0286505222320557
batch 100 loss value: 1.9839414358139038
batch 200 loss value: 1.978438138961792
batch 300 loss value: 1.9546740055084229
batch 400 loss value: 1.9086872339248657
acc:

In [12]:
# save model
torch.save(model.state_dict(), "data/model.pth")

In [13]:
model = NeuralNetwork()
model.load_state_dict(torch.load("data/model.pth"))

<All keys matched successfully>

In [14]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

In [18]:
X, y = test_data[0][0], test_data[0][1]

model.eval()
with torch.no_grad():
    pred = model(X)
    print(pred)
    print(pred[0].argmax(0))
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'prd:{predicted}, Actual:{actual}')

tensor([[-0.4030, -0.5586, -0.2213, -0.5873, -0.1592,  0.3175, -0.2769,  0.6456,
          0.5791,  1.2157]])
tensor(9)
prd:Ankle boot, Actual:Ankle boot


In [1]:
import matplotlib.pyplot as plt
# figure = plt.figure(figsize=(8, 8))
plt.imshow((test_data[1][0]).squeeze(), cmap="gray")
plt.show()

NameError: name 'test_data' is not defined